<a href="https://colab.research.google.com/github/hoanglam372000/Applied-Data-Science-Group-7/blob/nguyenthiman/7_12_TXL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 1.1.1 Thư viện:

In [1]:
!git clone https://github.com/hoanglam372000/Applied-Data-Science-Group-7.git

Cloning into 'Applied-Data-Science-Group-7'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 115 (delta 60), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (115/115), 4.86 MiB | 4.75 MiB/s, done.
Resolving deltas: 100% (60/60), done.


In [2]:
!pip install emoji
!pip install pyenchant
#!pip install pycontractions

     |████████████████████████████████| 170 kB 5.2 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=09b788a3cc77a79e2ce0a094eacbbbf68e4287df1373c9da461cd2e6990eac04
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji
     |████████████████████████████████| 55 kB 2.3 MB/s 


In [3]:
import math
import pandas as pd
import nltk
import numpy as np
import emoji
#import contractions
import re
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import scipy.stats as st
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from keras.layers import *
from keras.models import Model
from keras.callbacks import ModelCheckpoint
#print(stopwords.words('english'))
pd.set_option('display.max_colwidth', 200)
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('names')
names = nltk.corpus.names.words('male.txt')+nltk.corpus.names.words('female.txt')
names_lower = [name.lower() for name in names]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.


In [4]:
!pip install afinn

     |████████████████████████████████| 52 kB 780 kB/s 
  Created wheel for afinn: filename=afinn-0.1-py3-none-any.whl size=53448 sha256=e19b7feb22a2868983149467eac5a4544b941656fc2790760a565970b8efc763
  Stored in directory: /root/.cache/pip/wheels/9d/16/3a/9f0953027434eab5dadf3f33ab3298fa95afa8292fcf7aba75
Successfully built afinn


In [5]:
from afinn import Afinn
afinn = Afinn(language='en')

# TXL

In [8]:
# icon phổ biến
icon_df=pd.read_csv('icon.csv')
icon=icon_df['icon'].to_list()

In [9]:
stemmer = PorterStemmer()
# class xử lý vấn đề trùng ký tự trong từ, ví dụ yoooouuuu -> you, nooooo -> no.
#sử dụng pattern của regular expression, tuy nhiên phải quét thêm qua wordnet để tránh trường hợp như từ goose bị chuyển thành gose (mặc dù goose là từ đúng)
class RepeatReplacer(object):
    def __init__(self):
        self.repeat_regexp = re.compile(r'(\w*)(\w)\2(\w*)')
        self.repl = r'\1\2\3'
    def replace(self, word):
        if wordnet.synsets(word):
            return word
        repl_word = self.repeat_regexp.sub(self.repl, word)
        if repl_word != word:
            return self.replace(repl_word)
        else:
            return repl_word
char_repeat_correct = RepeatReplacer()
def preprocessing_word(word):
  if word != emoji.demojize(word):
    return emoji.demojize(word)
  #word = contractions.fix(word)
  #word = stemmer.stem(word)
  temp=[ "<html_tag>",'<URL>','<HIDDEN>','<tag>',"<hash_tag>"]
  if word in temp:
    return word
  #số theo từ
  if word.isdigit():  
    return '<NUMBER>'
  try: 
    float(word)
    return '<NUMBER>'
  except:
    pass

  #word=word.lower() # viết thườngthường
  #spell_corrector = Speller(lang='en') #sửa chính tả theo từng từ 
  #word=spell_corrector(word)

  # rút gọn theo từ
  #lemmatizer = WordNetLemmatizer()
  #wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
  #pos_tagged = nltk.pos_tag(word.split())
  
  #word = [lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged]
  #word = ' '.join(word)

  #word=word.translate(str.maketrans('', '', string.punctuation)) # chấm câu theo từ (vì tránh emoij)
  word = re.sub(pattern=r'[!"#$%&\'()*+,-\./:;<=>?@\[\]^_`{|}~0-9ï¿½]+', repl='', string=word)
  word = char_repeat_correct.replace(word)
  word = stemmer.stem(word)
  if (word in  names):
    return '<Name>'
  #word = stemmer.stem(word)
  word = word.lower()
  
  if word==''or word ==' ':
    return "<Special_Character>"
  
  if 'not' not in word and len(word.split())!=1:
    return word.split()[0]+'_'
  return word.replace(' ','_')

def preprocessing_text(tweet,sentiment):
   raw_text = str(tweet).strip().replace('\n', ' ')#.lower()
   # emoij
   raw_text=emoji.demojize(raw_text)
   e_r=r':\w+_\w+:'
   emoij_list=re.findall(e_r,raw_text)
   for e in emoij_list:
     raw_text=raw_text.replace(e,' ' + emoji.emojize(e) + ' ') # "hi❤️👍"
   if sentiment !=0: 
     raw_text = re.sub(pattern=r'(?<=\w)(\.){2,}?(?=\w)', repl=' ', string=raw_text)
   new_tweet = raw_text.split()
   len_=len(new_tweet)
   index2word = { w:new_tweet[w] for w in range(len_)}
   #parser = BeautifulSoup(raw_text, "html.parser")
   #raw_text = parser.get_text(separator = "<html_tag>")
   raw_text = re.sub(pattern=r'https?://\S+|www\.\S+', repl='<URL>', string=raw_text)
   raw_text = re.sub(pattern='\*{3,}?', repl='<HIDDEN>', string=raw_text)
   raw_text = re.sub(pattern=r'@\w+', repl='<tag>', string=raw_text)
   raw_text = re.sub(pattern=r'#\w+', repl='<hash_tag>', string=raw_text)
   
   word_list=raw_text.split()
   return index2word,dict(zip([*range(len_)],map(preprocessing_word,word_list)))

In [18]:
def preprocessing_string(tweet,sentiment):
   raw_text = str(tweet).strip().replace('\n', ' ')#.lower()
   # emoij
   raw_text=emoji.demojize(raw_text)
   e_r=r':\w+_\w+:'
   emoij_list=re.findall(e_r,raw_text)
   for e in emoij_list:
     raw_text=raw_text.replace(e,' ' + emoji.emojize(e) + ' ') # "hi❤️👍"
   if sentiment !=0: 
     raw_text = re.sub(pattern=r'(?<=\w)(\.){2,}?(?=\w)', repl=' ', string=raw_text)
   
  #  r = re.sub(r"[#!\"$%&\'*+,-\.?@\[\]():;/`{|}]*",'',w[1])
  #  if r == '' or r ==' ':
  #    raw_text=raw_text.replace(e,' ' + emoji.emojize(e) + ' ')
  #  else:
  #    preprocessed_tweet[w[0]] = r

   new_tweet = raw_text.split()
   len_=len(new_tweet)
   index2word = { w:new_tweet[w] for w in range(len_)}
   #parser = BeautifulSoup(raw_text, "html.parser")
   #raw_text = parser.get_text(separator = "<html_tag>")
   raw_text = re.sub(pattern=r'https?://\S+|www\.\S+', repl='<URL>', string=raw_text)
   raw_text = re.sub(pattern='\*{3,}?', repl='<HIDDEN>', string=raw_text)
   raw_text = re.sub(pattern=r'@\w+', repl='<tag>', string=raw_text)
   raw_text = re.sub(pattern=r'#\w+', repl='<hash_tag>', string=raw_text)
   
   word_list=raw_text.split()
   return word_list

preprocessing_string('hi:))',0) 

['hi:))']

In [27]:
w="hi:))"
s=r"[#!\"$%&\'*+,-\.?@\[\]():;/`{|}]*"


' [#!\\"$%&\\\'*+,-\\.?@\\[\\]():;/`{|}]*h [#!\\"$%&\\\'*+,-\\.?@\\[\\]():;/`{|}]*i [#!\\"$%&\\\'*+,-\\.?@\\[\\]():;/`{|}]* [#!\\"$%&\\\'*+,-\\.?@\\[\\]():;/`{|}]*'

In [13]:
# Test:
tweet = ' My:)) mom just texted.......me and told *ship* James that Rodney3 charlie hi❤️👍 wasï¿½ *** chasing?? and fireflies... in their :)) backyard. Awwwww I`m miss him!!! @hoanglam 123 ://    http://twitpic.com/66pn1'
tweet2 = 'nooooooooo'
index2word_temp,preprocessed_tweet_temp = preprocessing_text(tweet,0)
print('index2word: ',index2word_temp)
print('preprocessed_tweet: ',preprocessed_tweet_temp)

index2word:  {0: 'My:))', 1: 'mom', 2: 'just', 3: 'texted.......me', 4: 'and', 5: 'told', 6: '*ship*', 7: 'James', 8: 'that', 9: 'Rodney3', 10: 'charlie', 11: 'hi', 12: '❤️', 13: '👍', 14: 'wasï¿½', 15: '***', 16: 'chasing??', 17: 'and', 18: 'fireflies...', 19: 'in', 20: 'their', 21: ':))', 22: 'backyard.', 23: 'Awwwww', 24: 'I`m', 25: 'miss', 26: 'him!!!', 27: '@hoanglam', 28: '123', 29: '://', 30: 'http://twitpic.com/66pn1'}
preprocessed_tweet:  {0: 'my', 1: 'mom', 2: 'just', 3: 'textedm', 4: 'and', 5: 'told', 6: 'ship', 7: 'jame', 8: 'that', 9: 'rodney', 10: 'charli', 11: 'hi', 12: ':red_heart:', 13: ':thumbs_up:', 14: 'wa', 15: '<HIDDEN>', 16: 'chase', 17: 'and', 18: 'firefli', 19: 'in', 20: 'their', 21: '<Special_Character>', 22: 'backyard', 23: 'aw', 24: 'im', 25: 'miss', 26: 'him', 27: '<tag>', 28: '<NUMBER>', 29: '<Special_Character>', 30: '<URL>'}
